In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

def getyearmonth(row):
    return datetime.utcfromtimestamp(row['delivery_time.$date']/1000).strftime('%Y-%m')

f = open('temp/ido.json')
myjson = json.load(f)

orders = pd.json_normalize(myjson)
orders['venue_name_fixed'] = orders['venue_name'].str.extract(r'([^\\/|+]*)')
orders["year-month"] = orders.apply(lambda row: getyearmonth(row), axis=1)
orders = orders[orders['year-month'].str.startswith('2021')]

In [ ]:
best_rest = orders.groupby('venue_name_fixed').sum('total_price')
best_rest = best_rest.sort_values('total_price', ascending=False).total_price/100
best_rest


In [ ]:
orders.groupby('year-month').sum('total_price').total_price/100

In [ ]:
items = orders.explode('items')
items = items[items['items'].notna()]
items["item_name"] = items['items'].map(lambda x : x["name"])
items["item_count"] = items['items'].map(lambda x : x["count"])
items["item_total_price"] = items['items'].map(lambda x : x["end_amount"])

best_items = items.groupby('item_name').sum('item_total_price')
best_items.sort_values('item_total_price', ascending=False).item_total_price/100

In [ ]:
total_orders = orders['order_id'].count()
total_expenses = orders['total_price'].sum()/100
average_order = orders['total_price'].mean()/100
